In [9]:
# import libraries
# for processing data tables
import numpy as np
import pandas as pd

# for visualizations
import matplotlib.pyplot as plt
import seaborn as sns

# for sending requests to url and parsing
import requests
import json

import sys
sys.path.append('../')

# import useful cumtom functions and variables from my helper.py
from helper import request_data_kosis
from helper import dict_hl_name
from helper import map_hl
from helper import request_data_libsta

import configparser
config = configparser.ConfigParser()
config.read('../../config.ini')

['../../config.ini']

## Physiscian Density

In [16]:
df_phd_hl = request_data_kosis(url = config['phd']['hl_url'], target_period = '2021', variable_name = 'phd')
df_phd_ll = request_data_kosis(url = config['phd']['ll_url'], target_period = '2021', variable_name = 'phd')

In [17]:
df_phd_hl.head()

,C1,C1_NM,phd,period
0,11,서울특별시,4.7,2021
1,21,부산광역시,3.5,2021
2,22,대구광역시,3.6,2021
3,23,인천광역시,2.6,2021
4,24,광주광역시,3.7,2021


In [19]:
df_phd_ll['C1_hl'] = df_phd_ll['C1'].str[:2]
df_phd_ll.merge(df_phd_hl.loc[:,['C1','C1_NM']].rename(columns={'C1':'C1_hl','C1_NM':'hl'}), on='C1_hl', how = 'left')

,C1,C1_NM,phd,period,C1_hl,hl
0,11010,종로구,21.1,2021,11,서울특별시
1,11020,중구,10.2,2021,11,서울특별시
2,11030,용산구,3.6,2021,11,서울특별시
3,11040,성동구,3.6,2021,11,서울특별시
4,11050,광진구,3.8,2021,11,서울특별시
...,...,...,...,...,...,...
223,38380,함양군,1.8,2021,38,경상남도
224,38390,거창군,2.0,2021,38,경상남도
225,38400,합천군,2.2,2021,38,경상남도
226,39010,제주시,2.9,2021,39,제주특별자치도


## Number of Public Libraries

In [2]:
df_npl_hl = request_data_libsta(config['npl']['hl_url'], map_hl, dict_hl_name)

df_pop_hl = request_data_kosis(url = config['pop']['hl_url'], target_period = '2020', variable_name = 'pop')
df_pop_hl['C1_NM'] = df_pop_hl['C1_NM'].apply(lambda x: map_hl(x,dict_hl_name))

In [3]:
df_npl_hl = df_npl_hl.merge(df_pop_hl.loc[:,['C1_NM','pop']], on = 'C1_NM')
df_npl_hl['npl'] = df_npl_hl['npl'] / df_npl_hl['pop'] * 100000
df_npl_hl = df_npl_hl.drop('pop', axis = 1)

In [4]:
df_npl_hl.head()

,C1_NM,npl
0,강원도,16.408447
1,경기도,13.297450
2,경상남도,14.849793
3,경상북도,13.744984
4,광주광역시,33.030104


## Population Density

In [4]:
df_pop_hl = request_data_kosis(url = config['pop']['hl_url'], target_period = '2020', variable_name = 'pop')
df_ua_hl = request_data_kosis(url = config['ua']['hl_url'], target_period = '2020',variable_name = 'urban_area')

In [5]:
# Unfity the name of high-level regions
df_pop_hl['C1_NM'] = df_pop_hl['C1_NM'].apply(lambda x: map_hl(x,dict_hl_name))
df_ua_hl['C1_NM'] = df_ua_hl['C1_NM'].apply(lambda x: map_hl(x,dict_hl_name))

In [21]:
df_ua_hl.head()

,C1,C1_NM,urban_area,period
0,15315HJG001,서울특별시,605680193.0,2020
1,15315HJG002,부산광역시,940825055.0,2020
2,15315HJG003,대구광역시,799299015.0,2020
3,15315HJG004,인천광역시,529650659.0,2020
4,15315HJG005,광주광역시,480081726.0,2020


In [16]:
df_pop_hl = df_pop_hl.merge(df_ua_hl.loc[:,['C1_NM','urban_area']], on = 'C1_NM')
df_pop_hl['pd'] = df_pop_hl['pop'] /  df_pop_hl['urban_area']

In [20]:
df_pop_hl.sort_values(by = 'pop_density', ascending = False).head()

,C1,C1_NM,pop,period,urban_area,pop_density
0,1520213102303342A.01,서울특별시,9911088.0,2020,6.056802e+08,0.016364
3,1520213102303342A.04,인천광역시,3010476.0,2020,5.296507e+08,0.005684
8,1520213102303342A.09,경기도,13807158.0,2020,3.378712e+09,0.004087
1,1520213102303342A.02,부산광역시,3438710.0,2020,9.408251e+08,0.003655
4,1520213102303342A.05,광주광역시,1471385.0,2020,4.800817e+08,0.003065


## street_density

## land_allocated_to_streets